# Sheet properties

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Imports

In [6]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import array, logspace
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.transforms.common import FlattenTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
from toolbox.models.models import DecisionVotingClassifier, ScoreVotingClassifier
from toolbox.transforms.images import SpatialImageTransform
from toolbox.views.images import PatchViews
import warnings
warnings.filterwarnings('ignore') 

Using TensorFlow backend.


# Parameters

In [8]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])

# Sliding window

In [4]:
inputs_options = []
inputs_options.append((250,0))
inputs_options.append((500,0))

# Low level predictions

In [9]:
for size, overlap in inputs_options:
    inputs = IO.load(f'Spatial_{size}_{overlap}.pickle')
    
    PatchViews.display(inputs,Tools.PREDICTION, settings, label_encoder, index=0)    
    PatchViews.display(inputs,Tools.PROBABILITY, settings, label_encoder, index=0)

KeyError: 'Prediction'